In [71]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


# Create a producer object
# Ip can be different in your case

def Convert(a):
    it = iter(a)
    res_dct = dict(zip(it, it))
    return res_dct
df1=pd.DataFrame(columns=['İlan No:', 'İlan Tarihi:', 'Marka:', 'Seri:', 'Model:', 'Yıl:',
       'Yakıt Tipi:', 'Vites Tipi:', 'Motor Hacmi:', 'Motor Gücü:',
       'Kilometre:', 'Boya-değişen:', 'Takasa Uygun:', 'Kimden:'])
for page in range(1,50):
          
      
        url="https://www.arabam.com/ikinci-el/otomobil?page="+str(page)

        r=requests.get(url)
        soup=BeautifulSoup(r.content,'html.parser')
        table=soup.find("table",attrs={"class":"table listing-table w100 border-grey2"})
        cars=table.find_all("a",attrs={"class":"listing-text-new word-break val-middle color-black2018"})
        #price_table=table.find_all("td",attrs={"class":"pl8 pr8 tac pr"})
        #price=[]
        #for i in price_table:
            #price.append(i.text)
        
        for car in cars:
            car_link=[]
            car_Link=car.get("href")
            car_Link="https://www.arabam.com/"+car_Link
            car_link.append(car_Link)
            #car_r=requests.get(car_Link)
            soup = BeautifulSoup(requests.get(car_Link).content, 'html.parser')
            table1=soup.find_all("span",attrs={"class":"bli-particle"})
            att=[]
            
            for i in table1:
                att.append(i.text)
            price=soup.find("div",attrs={"class":"mb8"}).text
                
               
            data=Convert(att)
            df=pd.DataFrame.from_dict(data, orient="index").T
            df["car_link:"]=car_link
            df["price"]=price
            df1=pd.concat([df1,df],ignore_index=True,join="outer")

df1=df1.loc[:,['İlan No:', 'İlan Tarihi:', 'Marka:', 'Seri:', 'Model:', 'Yıl:','Yakıt Tipi:', 'Vites Tipi:', 'Motor Hacmi:', 'Motor Gücü:',"Kilometre:",'car_link:',"price"]]
df1.drop_duplicates(subset=['İlan No:'], keep='last',inplace=True)
df1.to_csv('arabam.csv')



    

In [70]:
df1.head()

,İlan No:,İlan Tarihi:,Marka:,Seri:,Model:,Yıl:,Yakıt Tipi:,Vites Tipi:,Motor Hacmi:,Motor Gücü:,Kilometre:,car_link:,price
0,16251517,23 Aralık 2020,Volkswagen,Passat Variant,1.6 TDi BlueMotion Comfortline,2012,Dizel,Yarı Otomatik,1598 cc,105 hp,70.000 km,https://www.arabam.com//ilan/sahibinden-satili...,200.000 TL
1,15943402,22 Aralık 2020,Volkswagen,Polo,1.4 Comfortline,2013,LPG & Benzin,Düz,1390 cc,85 hp,127.000 km,https://www.arabam.com//ilan/galeriden-satilik...,122.500 TL
2,15788492,22 Aralık 2020,Chevrolet,Cruze,1.6 LS,2009,LPG & Benzin,Düz,1598 cc,113 hp,226.000 km,https://www.arabam.com//ilan/galeriden-satilik...,85.000 TL
3,16250683,22 Aralık 2020,Renault,Megane,1.5 dCi Touch,2020,Dizel,Yarı Otomatik,1461 cc,115 hp,0 km,https://www.arabam.com//ilan/galeriden-satilik...,277.000 TL
4,16250505,22 Aralık 2020,Volkswagen,VW CC,1.4 TSi,2013,Benzin,Yarı Otomatik,1390 cc,160 hp,49.000 km,https://www.arabam.com//ilan/galeriden-satilik...,192.500 TL
